# Ansible Adhoc Kommandos

Zunächst muss Ansible auf einem Host installiert sein.

http://docs.ansible.com/ansible/intro_installation.html

Danch können einfache Adhoc Kommandos ausgeführt werden.

http://docs.ansible.com/ansible/intro_getting_started.html

In [1]:
history -c

Löscht die Bash-History. Siehe https://wiki.ubuntuusers.de/Bash/#History

Dann kann getestet werden ob ansible installiert ist.

In [2]:
ansible --version

ansible 2.1.2.0
  config file = 
  configured module search path = Default w/o overrides


Update ansible via pip.

Wegen pip siehe: https://docs.python.org/2/installing/

In [3]:
sudo pip install --upgrade ansible

The directory '/home/vagrant/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/vagrant/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
Requirement already up-to-date: ansible in /usr/local/lib/python2.7/dist-packages
Requirement already up-to-date: PyYAML in /usr/local/lib/python2.7/dist-packages (from ansible)
    100% |████████████████████████████████| 481kB 864kB/s 
Requirement already up-to-date: jinja2 in /usr/local/lib/python2.7/dist-packages (from ansible)
Requirement already up-to-date: paramiko in /usr/local/lib/python2.7/dist-packages (from ansible)
Requirement already up-to-date: pycrypto>=2.6 in /usr/local/lib/python2.7/

In [4]:
ansible --version

ansible 2.1.2.0
  config file = 
  configured module search path = Default w/o overrides


Einrichten einer Inventorydatei.

http://docs.ansible.com/ansible/intro_inventory.html

In [5]:
ls -lachi /etc/ansible/
sudo cp /vagrant/hosts /etc/ansible

insgesamt 12K
525129 drwxr-xr-x   3 root root 4,0K Sep 21 17:17 .
262145 drwxr-xr-x 115 root root 4,0K Okt 17 08:07 ..
525175 drwxr-xr-x  10 root root 4,0K Okt 12 20:34 roles


In [6]:
cat /etc/ansible/hosts

[ctl]
127.0.0.1   ansible_connection=local

[www]
192.168.60.11
192.168.60.12
192.168.60.13

[db]
192.168.60.21
192.168.60.22

[lb]
192.168.60.2

[backend:children]
www
db

# Point of entry
[poe:children]
lb
www


In [8]:
ansible ctl -m ping

127.0.0.1 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


Erstellen eines ssh-key ohne Nachfrage. Ein evtl. existierender Schlüssel wird gelöscht.

ACHTUNG: Nicht für produktiv Systeme. Hier besser mit Variablen arbeiten.

Wegen ENV Siehe auch: https://12factor.net/de/

ToDO. Hier mit ENV zeigen.

SSH_PASSPHRASE=geheim

In [9]:
if [ -f ~/.ssh/id_rsa ]; then rm ~/.ssh/id_rsa*; fi && ssh-keygen -t rsa -f ~/.ssh/id_rsa -N geheim

Generating public/private rsa key pair.
Your identification has been saved in /home/vagrant/.ssh/id_rsa.
Your public key has been saved in /home/vagrant/.ssh/id_rsa.pub.
The key fingerprint is:
SHA256:KJ1V10SR4xSUA6dhrKkjrYB6CNwiwYPg8rewNjWZqVY vagrant@ctl
The key's randomart image is:
+---[RSA 2048]----+
|          ..=*B= |
|.        . o.+B  |
|=       .  o.o o |
|++   . +  o   .  |
|ooo..++.S.       |
|oo+.E.. +        |
|oo.B + o .       |
|o B . .          |
| + .             |
+----[SHA256]-----+


Automatisches verteilen mit ssh-cop-id

http://serverfault.com/questions/306541/automating-ssh-copy-id

Achtung: Der Host-Key muss als erstes akzeptiert werden.

Can I automatically add a new host to known_hosts?

http://serverfault.com/questions/132970/can-i-automatically-add-a-new-host-to-known-hosts

Nur etwas, wenn dem Netz vetraut wird. NICHT FÜR PRODUKTIV SYSTEME!

In [10]:
ssh-keyscan -H 192.168.60.2 >> ~/.ssh/known_hosts

# 192.168.60.2:22 SSH-2.0-OpenSSH_7.2p2 Ubuntu-4ubuntu2.1
# 192.168.60.2:22 SSH-2.0-OpenSSH_7.2p2 Ubuntu-4ubuntu2.1
# 192.168.60.2:22 SSH-2.0-OpenSSH_7.2p2 Ubuntu-4ubuntu2.1


In [11]:
cat ~/.ssh/known_hosts

|1|plnagrukktq51R6s86RfFo5LpWc=|Y3qGwPjRHth9gb7tBlbEl24C/jg= ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDU171qwSgbEtEal2fCozSU7wsNRzA2H/DuUaLku6xRazRkfQZnFHcDn5pFk/GhHLTCwDRXz7nnnbBFFJK3jr03AsOr6rtqJuojQEIh1Yz1wKI4Dvvawteb8UnveKcef7OGs6a2nphGC+ZYX8txu+gdSF9HNVxGbUyFtbNeKRpAPCRgAkHfh1stxwPK8BQiaPQbEHSWzPer1G38Qsq1QzbKXWrizU057i5pw38cdRQ8CaVYe9u+rUnNEOfGjpESh06lb1vdZDEmily7D0jWDW7+Tw7hbr3uw+wUiJaVEGrBbQQotIqcrWTz6AqrwXKdZpklJZkcPx5JGS3Ym13AUBDB
|1|E2JMOtS40Z89qlsiSDy99ynMaTY=|oIN/lWhb5VoHoPvciSUbAlkpLjA= ecdsa-sha2-nistp256 AAAAE2VjZHNhLXNoYTItbmlzdHAyNTYAAAAIbmlzdHAyNTYAAABBBOUj8l+Ejj53rTy5+9bMcbINLRZ6UFxaK6fRE6uklH7YvPh08DepyspTONUWuwwYSwJuHBldP8Joafx5jNuW2sI=
|1|WX4OU3NhdMdxeMIsXTI69q932pc=|s9UgWihQrQxhQM4P6004v59Zzb8= ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIAc42trNYnv58q3PiJSJ+lLhkHY9NOX4kVrrJdC8fJLf


SSH Schlüssel verteilen ohne Passwortabfrage.

In [12]:
sudo apt-get update
sudo apt-get install -y sshpass

OK:1 http://de.archive.ubuntu.com/ubuntu xenial InRelease
OK:2 http://ppa.launchpad.net/chronitis/jupyter/ubuntu xenial InRelease
OK:3 http://de.archive.ubuntu.com/ubuntu xenial-updates InRelease
OK:4 http://de.archive.ubuntu.com/ubuntu xenial-backports InRelease
OK:5 http://ppa.launchpad.net/nginx/stable/ubuntu xenial InRelease
Ign:6 http://repo.mongodb.org/apt/ubuntu trusty/mongodb-org/3.2 InRelease
OK:7 http://security.ubuntu.com/ubuntu xenial-security InRelease
OK:8 http://ppa.launchpad.net/nijel/phpmyadmin/ubuntu xenial InRelease
OK:9 http://repo.mongodb.org/apt/ubuntu trusty/mongodb-org/3.2 Release
OK:10 https://apt.dockerproject.org/repo ubuntu-xenial InRelease
OK:11 https://deb.nodesource.com/node_4.x xenial InRelease




Die folgenden NEUEN Pakete werden installiert:
  sshpass
0 aktualisiert, 1 neu installiert, 0 zu entfernen und 8 nicht aktualisiert.
Es müssen 10,5 kB an Archiven heruntergeladen werden.
Nach dieser Operation werden 56,3 kB Plattenplatz zusätzlich benutzt.
Hol

In [13]:
echo 'vagrant' > passwort
sshpass -f passwort ssh-copy-id vagrant@192.168.60.2

/usr/bin/ssh-copy-id: INFO: Source of key(s) to be installed: "/home/vagrant/.ssh/id_rsa.pub"
/usr/bin/ssh-copy-id: INFO: attempting to log in with the new key(s), to filter out any that are already installed
/usr/bin/ssh-copy-id: INFO: 1 key(s) remain to be installed -- if you are prompted now it is to install the new keys

Number of key(s) added: 1

Now try logging into the machine, with:   "ssh 'vagrant@192.168.60.2'"
and check to make sure that only the key(s) you wanted were added.



Da wir in Jupyter bzw. in einem Skript keine passphrase eingeben können, hier ein Workaround mit dem Kommando expect.

https://wiki.ubuntuusers.de/Tcl/#Installation-ueber-Download

In [14]:
sudo apt-get install -y expect




Die folgenden zusätzlichen Pakete werden Installiert
  tcl-expect
Die folgenden NEUEN Pakete werden installiert:
  expect tcl-expect
0 aktualisiert, 2 neu installiert, 0 zu entfernen und 8 nicht aktualisiert.
Es müssen 241 kB an Archiven heruntergeladen werden.
Nach dieser Operation werden 538 kB Plattenplatz zusätzlich benutzt.
Holen:1 http://de.archive.ubuntu.com/ubuntu xenial/universe amd64 tcl-expect amd64 5.45-7 [104 kB]
Holen:2 http://de.archive.ubuntu.com/ubuntu xenial/universe amd64 expect amd64 5.45-7 [137 kB]
Es wurden 241 kB in 0 s geholt (548 kB/s).
debconf: kann Oberfläche nicht initialisieren: Dialog
debconf: (Die Dialog-Oberfläche funktioniert nicht auf einem Dumb-Terminal, einem Emacs-Shellbuffer oder ohne ein steuerndes Terminal.)
debconf: greife zurück auf die Oberfläche: Readline
Vormals nicht ausgewähltes Paket tcl-expect:amd64 wird gewählt.
(Lese Datenbank ... 243038 Dateien und Verzeichnisse sind derzeit installiert.)
Vorbereitung zum Entpacken von .../tcl-expe

In [15]:
cp /vagrant/ssh-add-passphrase.sh ./
chmod u+x ssh-add-passphrase.sh

In [16]:
cat ssh-add-passphrase.sh

#!/usr/bin/expect -f
spawn ssh-add /home/vagrant/.ssh/id_rsa
expect "Enter passphrase for /home/vagrant/.ssh/id_rsa:"
send "geheim\n";
interact


Setzen der ssh-agent Umgebung für die jetzige Bashsitzung. Dies muss für jede neue Bashsitzung erneut durchgeführt werden.

In [17]:
eval `ssh-agent -s` > /dev/null
./ssh-add-passphrase.sh


Enter passphrase for /home/vagrant/.ssh/id_rsa: 
Identity added: /home/vagrant/.ssh/id_rsa (/home/vagrant/.ssh/id_rsa)


In [18]:
env

SSH_AGENT_PID=6823
XDG_SESSION_ID=c1
SHELL=/bin/bash
USER=vagrant
LS_COLORS=
SSH_AUTH_SOCK=/tmp/ssh-RRBJIWd9Q84l/agent.6822
JPY_PARENT_PID=1269
PAGER=cat
MAIL=/var/mail/vagrant
PATH=/home/vagrant/.gem/ruby/2.3.0/bin:/home/vagrant/bin:/home/vagrant/.local/bin:/usr/local/lib/npm/bin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/snap/bin
NPM_CONFIG_PREFIX=/usr/local/lib/npm
PWD=/home/vagrant/projects
LANG=de_DE.UTF-8
NODE_PATH=/usr/lib/nodejs:/usr/lib/node_modules:/usr/share/javascript:/usr/local/lib/npm/lib/node_modules
PS1=[PEXP\[\]ECT_PROMPT>
SHLVL=2
HOME=/home/vagrant
LOGNAME=vagrant
LESSOPEN=| /usr/bin/lesspipe %s
XDG_RUNTIME_DIR=/run/user/1000
LESSCLOSE=/usr/bin/lesspipe %s %s
_=/usr/bin/env


In [20]:
export

declare -x HOME="/home/vagrant"
declare -x JPY_PARENT_PID="1269"
declare -x LANG="de_DE.UTF-8"
declare -x LESSCLOSE="/usr/bin/lesspipe %s %s"
declare -x LESSOPEN="| /usr/bin/lesspipe %s"
declare -x LOGNAME="vagrant"
declare -x LS_COLORS=""
declare -x MAIL="/var/mail/vagrant"
declare -x NODE_PATH="/usr/lib/nodejs:/usr/lib/node_modules:/usr/share/javascript:/usr/local/lib/npm/lib/node_modules"
declare -x NPM_CONFIG_PREFIX="/usr/local/lib/npm"
declare -x OLDPWD
declare -x PAGER="cat"
declare -x PATH="/home/vagrant/.gem/ruby/2.3.0/bin:/home/vagrant/bin:/home/vagrant/.local/bin:/usr/local/lib/npm/bin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/snap/bin"
declare -x PS1="[PEXP\\[\\]ECT_PROMPT>"
declare -x PWD="/home/vagrant/projects"
declare -x SHELL="/bin/bash"
declare -x SHLVL="2"
declare -x SSH_AGENT_PID="6823"
declare -x SSH_AUTH_SOCK="/tmp/ssh-RRBJIWd9Q84l/agent.6822"
declare -x USER="vagrant"
declare -x XDG_RUNTIME_DIR="/run/user/1000"
declare -x XDG_SESSION_ID="c1"


In [19]:
ssh-add -l

2048 SHA256:KJ1V10SR4xSUA6dhrKkjrYB6CNwiwYPg8rewNjWZqVY /home/vagrant/.ssh/id_rsa (RSA)


In [21]:
ansible lb -m ping

192.168.60.2 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


For Schleife in der Bash. Achtung erst den Maschienen-Fingerprint akzeptieren und dann den Schlüssel verteilen.

Mehr zu Bash-Skripte unter:

* https://wiki.ubuntuusers.de/Shell/Bash-Skripting-Guide_f%C3%BCr_Anf%C3%A4nger/

In [22]:
for i in 11 12 13 21 22
do 
    ssh-keyscan -H 192.168.60.${i} >> ~/.ssh/known_hosts
    sshpass -f passwort ssh-copy-id vagrant@192.168.60.${i}
done

# 192.168.60.11:22 SSH-2.0-OpenSSH_7.2p2 Ubuntu-4ubuntu2.1
# 192.168.60.11:22 SSH-2.0-OpenSSH_7.2p2 Ubuntu-4ubuntu2.1
# 192.168.60.11:22 SSH-2.0-OpenSSH_7.2p2 Ubuntu-4ubuntu2.1
/usr/bin/ssh-copy-id: INFO: attempting to log in with the new key(s), to filter out any that are already installed
/usr/bin/ssh-copy-id: INFO: 1 key(s) remain to be installed -- if you are prompted now it is to install the new keys

Number of key(s) added: 1

Now try logging into the machine, with:   "ssh 'vagrant@192.168.60.11'"
and check to make sure that only the key(s) you wanted were added.

# 192.168.60.12:22 SSH-2.0-OpenSSH_7.2p2 Ubuntu-4ubuntu2.1
# 192.168.60.12:22 SSH-2.0-OpenSSH_7.2p2 Ubuntu-4ubuntu2.1
# 192.168.60.12:22 SSH-2.0-OpenSSH_7.2p2 Ubuntu-4ubuntu2.1
/usr/bin/ssh-copy-id: INFO: attempting to log in with the new key(s), to filter out any that are already installed
/usr/bin/ssh-copy-id: INFO: 1 key(s) remain to be installed -- if you are prompted now it is to install the new keys

Number of key

In [25]:
ansible all -m ping

127.0.0.1 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
192.168.60.21 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
192.168.60.2 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
192.168.60.11 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
192.168.60.12 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
192.168.60.13 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
192.168.60.22 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


In [24]:
history 

    1  echo $?
    2  ansible --version
    3  echo $?
    4  sudo pip install --upgrade ansible
    5  echo $?
    6  ansible --version
    7  echo $?
    8  ls -lachi /etc/ansible/
    9  sudo cp /vagrant/hosts /etc/ansible
   10  echo $?
   11  cat /etc/ansible/hosts
   12  echo $?
   13  ansible local -m ping
   14  echo $?
   15  ansible ctl -m ping
   16  echo $?
   17  if [ -f ~/.ssh/id_rsa ]; then rm ~/.ssh/id_rsa*; fi && ssh-keygen -t rsa -f ~/.ssh/id_rsa -N geheim
   18  echo $?
   19  ssh-keyscan -H 192.168.60.2 >> ~/.ssh/known_hosts
   20  echo $?
   21  cat ~/.ssh/known_hosts
   22  echo $?
   23  sudo apt-get update
   24  sudo apt-get install -y sshpass
   25  echo $?
   26  echo 'vagrant' > passwort
   27  sshpass -f passwort ssh-copy-id vagrant@192.168.60.2
   28  echo $?
   29  sudo apt-get install -y expect
   30  echo $?
   31  cp /vagrant/ssh-add-passphrase.sh ./
   32  chmod u+x ssh-add-passphrase.sh
   33  echo $?
   34  cat ssh-add-passphrase.sh
   35  echo $?
 

__Aufgabe__: Da wir öfters ein ```bash vagrant destroy``` ausführen. Wäre es sinnvoll ein Skript für das automatische Einrichten von Ansible und ssh zu haben. Erstellen Sie dieses Skript.

Der Ansible Test funktioniert. Somit können wir mit den Playbooks weiter machen.